In [ ]:
import pandas as pd

df_downloaded = pd.read_csv("/content/downloaded_data_2.csv")
display(df_downloaded.head())

ModuleNotFoundError: No module named 'gdown'

In [ ]:
df = df_downloaded.head(100000)

# scape condo

In [ ]:
import requests
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# ---------------------------------------------------------
# PART 1: Fetch External Data (Condo/Apartment from OSM)
# ---------------------------------------------------------
print("1. Fetching Condo Data from OpenStreetMap API...")

overpass_url = "http://overpass-api.de/api/interpreter"
# Bounding Box ครอบคลุมพื้นที่ กทม. และปริมณฑล
bbox = "(13.45, 100.30, 14.05, 101.00)"

# Query: หาตึกที่เป็นที่พักอาศัยและมีชื่อระบุ
query = f"""
[out:json][timeout:60];
(
  node["residential"="condominium"]{bbox};
  way["residential"="condominium"]{bbox};
  node["building"="apartments"]{bbox};
  way["building"="apartments"]{bbox};
  node["name"~"Condo|Residence|Mansion|Apartment",i]{bbox};
);
out center;
"""

try:
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()

    condo_list = []
    for item in data.get('elements', []):
        # ดึงพิกัด (รองรับทั้ง node และ way)
        lat = item.get('lat') or item.get('center', {}).get('lat')
        lon = item.get('lon') or item.get('center', {}).get('lon')

        # ดึงชื่อ (พยายามเอาภาษาไทยก่อน ถ้าไม่มีเอาอังกฤษ)
        tags = item.get('tags', {})
        name = tags.get('name:th') or tags.get('name:en') or tags.get('name')

        if lat and lon and name:
            condo_list.append({
                'condo_name': name,
                'lat': lat,
                'lon': lon
            })

    df_condo = pd.DataFrame(condo_list)
    print(f"   -> Raw data fetched: {len(df_condo)} records")

except Exception as e:
    print(f"Error fetching data: {e}")
    # ถ้า API พัง ให้สร้าง df_condo เปล่าๆ เพื่อกันโค้ด Error
    df_condo = pd.DataFrame(columns=['condo_name', 'lat', 'lon'])

# ---------------------------------------------------------
# PART 2: Clean External Data
# ---------------------------------------------------------
print("2. Cleaning Condo Data...")

if not df_condo.empty:
    # 2.1 ลบข้อมูลซ้ำ (ชื่อเดียวกันเอาไว้อันเดียว)
    df_condo.drop_duplicates(subset=['condo_name'], keep='first', inplace=True)

    # 2.2 กรอง Noise (ตัดสำนักงานขาย, โชว์รูม, ร้านค้า)
    noise_keywords = ['Sale Gallery', 'Office', 'Showroom', 'สำนักงานขาย', 'ร้าน', 'Shop', '7-Eleven', 'Market']
    pattern = '|'.join(noise_keywords)
    df_condo = df_condo[~df_condo['condo_name'].str.contains(pattern, case=False, na=False)]

    # 2.3 แปลงพิกัดเป็นตัวเลข
    df_condo['lat'] = pd.to_numeric(df_condo['lat'], errors='coerce')
    df_condo['lon'] = pd.to_numeric(df_condo['lon'], errors='coerce')

    print(f"   -> Cleaned data remaining: {len(df_condo)} records")
else:
    print("   -> No condo data available to process.")

# ---------------------------------------------------------
# PART 3: Integrate with Traffy Data & Calculate Features
# ---------------------------------------------------------
print("3. Calculating Distance and Density...")

# ใช้ตัวแปร clean_organization จากขั้นตอนก่อนหน้า
target_df = df.copy()
target_df['province'] = target_df['province'].str.strip()
target_df = target_df[target_df['province'] == 'กรุงเทพมหานคร']

# แปลงพิกัด Traffy เป็นตัวเลข
target_df['latitude'] = pd.to_numeric(target_df['latitude'], errors='coerce')
target_df['longitude'] = pd.to_numeric(target_df['longitude'], errors='coerce')
target_df = target_df.dropna(subset=['latitude', 'longitude'])

if not df_condo.empty:
    # เตรียมพิกัดสำหรับ cKDTree
    condo_coords = df_condo[['lat', 'lon']].values
    traffy_coords = target_df[['latitude', 'longitude']].values

    # สร้าง Tree ครั้งเดียว ใช้ได้ทั้งสองงาน
    tree = cKDTree(condo_coords)

    # --- TASK A: หาคอนโดที่ "ใกล้ที่สุด" (Distance) ---
    # k=1 คือเอาแค่ 1 จุดที่ใกล้ที่สุด
    dist_deg, indices = tree.query(traffy_coords, k=1)

    # แปลงองศาเป็น Km (1 องศา ≈ 111.12 km)
    target_df['dist_to_nearest_condo_km'] = dist_deg * 111.12
    target_df['nearest_condo_name'] = df_condo.iloc[indices]['condo_name'].values

    # --- TASK B: นับจำนวนคอนโดในรัศมี 1 กม. (Density) ---
    # รัศมี 1 km แปลงเป็นองศา
    radius_deg = 1 / 111.12

    # query_ball_point จะคืนค่าเป็น list ของ index ที่อยู่ในวงกลม
    indices_in_radius = tree.query_ball_point(traffy_coords, r=radius_deg)

    # นับจำนวนสมาชิกใน list
    target_df['condo_count_1km'] = [len(x) for x in indices_in_radius]

    print("   -> Calculation Completed!")
else:
    print("   -> Skipping calculation (No condo data).")

1. Fetching Condo Data from OpenStreetMap API...
   -> Raw data fetched: 1164 records
2. Cleaning Condo Data...
   -> Cleaned data remaining: 1041 records
3. Calculating Distance and Density...
   -> Calculation Completed!


# scape สภาพอากาศ

In [ ]:
import requests
import pandas as pd

def get_historical_weather(start_date, end_date, lat=13.7563, lon=100.5018):
    """
    ดึงข้อมูลจาก Open-Meteo (Archive API)
    Default Location: Bangkok (lat=13.7563, lon=100.5018)
    """
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "weather_code", # ดึงรหัสสภาพอากาศรายวัน
        "timezone": "Asia/Bangkok"
    }

    response = requests.get(url, params=params)
    data = response.json()

    # สร้าง DataFrame จากข้อมูลดิบ
    df = pd.DataFrame({
        "date": pd.to_datetime(data["daily"]["time"]),
        "weather_code": data["daily"]["weather_code"]
    })

    return df

def map_weather_condition(code):
    """
    แปลง WMO Weather Code เป็น 5 กลุ่มหลักตามโจทย์
    """
    # Group 1: ฟ้าใส / แดดออก
    if code in [0]:
        return "ฟ้าใส"
    # Group 2: มีเมฆ (บางส่วน หรือ มาก)
    elif code in [1, 2, 3]:
        return "มีเมฆ"
    # Group 3: หมอก (จัดเป็นกลุ่มอื่นๆ หรือรวมกับเมฆก็ได้)
    elif code in [45, 48]:
        return "หมอก"
    # Group 4: ฝนตก (ละอองฝน, ฝนตกหนัก, ฝนเยือกแข็ง)
    elif code in [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 80, 81, 82]:
        return "ฝนตก"
    # Group 5: พายุ / หิมะ (รวมหิมะไว้เผื่อกรณีต่างประเทศ แต่ไทยหลักๆ คือพายุ)
    elif code in [95, 96, 99]:
        return "พายุฝน"
    # กรณีอื่นๆ (เช่น หิมะ code 71-77)
    else:
        return "อื่นๆ"

# --- การใช้งาน ---

# 1. กำหนดวันและดึงข้อมูล
start = "2021-09-19"
end = "2022-06-08"

# ดึงข้อมูล (ตัวอย่างพิกัด กรุงเทพฯ)
df_weather = get_historical_weather(start, end)

# 2. แปลง Code เป็นข้อความ (Map values)
df_weather['สภาพอากาศ'] = df_weather['weather_code'].apply(map_weather_condition)

# 3. จัด Format ให้เหลือแค่ 1 Column (โดยให้ Date เป็น Index เพื่อความสวยงาม)
final_df = df_weather.set_index('date')[['สภาพอากาศ']]

# แสดงผลลัพธ์
print(f"ดึงข้อมูลวันที่: {start} ถึง {end}")
print("-" * 30)
print(final_df)

# เช็คจำนวนกลุ่มสภาพอากาศที่มี
print("\nสรุปจำนวนประเภทสภาพอากาศที่พบ:")
print(final_df['สภาพอากาศ'].value_counts())

ดึงข้อมูลวันที่: 2021-09-19 ถึง 2022-06-08
------------------------------
           สภาพอากาศ
date                
2021-09-19      ฝนตก
2021-09-20      ฝนตก
2021-09-21      ฝนตก
2021-09-22      ฝนตก
2021-09-23      ฝนตก
...              ...
2022-06-04      ฝนตก
2022-06-05      ฝนตก
2022-06-06      ฝนตก
2022-06-07      ฝนตก
2022-06-08      ฝนตก

[263 rows x 1 columns]

สรุปจำนวนประเภทสภาพอากาศที่พบ:
สภาพอากาศ
ฝนตก     175
มีเมฆ     86
ฟ้าใส      2
Name: count, dtype: int64


In [ ]:
final_df['สภาพอากาศ'].head()

,สภาพอากาศ
date,
2021-09-19,ฝนตก
2021-09-20,ฝนตก
2021-09-21,ฝนตก
2021-09-22,ฝนตก
2021-09-23,ฝนตก


In [ ]:
min_timestamp = target_df['timestamp'].min()
max_timestamp = target_df['timestamp'].max()

print(f"Minimum Timestamp: {min_timestamp}")
print(f"Maximum Timestamp: {max_timestamp}")

Minimum Timestamp: 2021-09-19 14:56:08.924992+00
Maximum Timestamp: 2022-09-13 07:43:03.311497+00


In [ ]:
target_df['timestamp'] = pd.to_datetime(target_df['timestamp'], errors='coerce')
target_df['date'] = target_df['timestamp'].dt.normalize().dt.tz_localize(None)
target_df = pd.merge(target_df, final_df, left_on='date', right_index=True, how='left')

print(target_df[['timestamp', 'date', 'สภาพอากาศ']].head())

                         timestamp       date สภาพอากาศ
0 2021-09-19 14:56:08.924992+00:00 2021-09-19      ฝนตก
1 2021-09-26 05:03:52.594898+00:00 2021-09-26      ฝนตก
2 2021-12-09 12:29:08.408763+00:00 2021-12-09     มีเมฆ
3 2021-12-18 14:50:52.437512+00:00 2021-12-18     มีเมฆ
4 2021-12-22 10:15:33.294829+00:00 2021-12-22     มีเมฆ


In [ ]:
target_df.head()

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,type 3,organization_1,organization_2,organization_3,dist_to_nearest_condo_km,nearest_condo_name,condo_count_1km,cluster,date,สภาพอากาศ
0,1,2021-CGPMUN,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00:00,...,NaN,เขตประเวศ,ฝ่ายโยธา เขตประเวศ,NaN,1.180890,South Park Apartment,0,1,2021-09-19,ฝนตก
1,2,2021-7XATFA,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00:00,...,NaN,เขตสาทร,NaN,NaN,0.134856,Silom Suite,19,1,2021-09-26,ฝนตก
2,4,2021-DVEWYM,ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00:00,...,NaN,เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว,NaN,1.210388,Plum condo Chokchai4 (พลัมคอนโด โชคชัย4),0,1,2021-12-09,มีเมฆ
3,7,2021-8N9ZP8,คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 14:50:52.437512+00:00,...,NaN,เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะอาดฯ เขตประเวศ,0.291524,Supalai Park (Condos),2,3,2021-12-18,มีเมฆ
4,9,2021-8BTWZB,ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00:00,...,NaN,เขตประเวศ,ฝ่ายโยธา เขตประเวศ,NaN,1.457128,Supalai Park (Condos),0,3,2021-12-22,มีเมฆ


# scape ความหนาแน่นคน

In [ ]:
# ==============================================================================
# DATA PIPELINE (FIXED): Scrape Bangkok Population with Headers
# ==============================================================================
import pandas as pd
import requests

# 1. Scrape Data from Wikipedia (with Headers)
print("1. Scraping population data from Wikipedia...")
url = "https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok"

# [FIX] เพิ่ม User-Agent เพื่อไม่ให้โดนบล็อก 403
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    # ยิง Request แบบมี Header
    response = requests.get(url, headers=headers)
    response.raise_for_status() # เช็คว่าผ่านไหม (200 OK)

    # อ่านตารางจากเนื้อหา HTML ที่ได้มา
    tables = pd.read_html(response.text)

    # ตารางแรกคือตารางเขต
    df_pop = tables[0]

    # Rename Columns
    # (ชื่อคอลัมน์ Wiki บางทีมี [1] ติดมา ต้องใช้ .str.contains หรือ rename แบบกว้างๆ)
    print("   -> Raw columns:", df_pop.columns.tolist())

    # Map ชื่อคอลัมน์ใหม่ (สังเกตตำแหน่งคอลัมน์จากหน้าเว็บ)
    # ปกติ: District (Khet) | Thai | Population | ...
    df_pop = df_pop.rename(columns={
        df_pop.columns[0]: 'district_en', # District (Khet)
        df_pop.columns[1]: 'district',    # Thai name
        df_pop.columns[2]: 'population',  # Population
        df_pop.columns[4]: 'area_sqkm'    # Area
    })

    # Clean Data
    # ลบลูกน้ำ, ลบ footnote [1], แปลงเป็นตัวเลข
    df_pop['population'] = df_pop['population'].astype(str).str.replace(r'\[.*\]', '', regex=True).str.replace(',', '')
    df_pop['area_sqkm'] = df_pop['area_sqkm'].astype(str).str.replace(r'\[.*\]', '', regex=True).str.replace(',', '')

    df_pop['population'] = pd.to_numeric(df_pop['population'], errors='coerce')
    df_pop['area_sqkm'] = pd.to_numeric(df_pop['area_sqkm'], errors='coerce')

    # สร้าง Feature: ความหนาแน่นประชากร (คน/ตร.กม.)
    df_pop['pop_density'] = df_pop['population'] / df_pop['area_sqkm']

    print(f"   -> Scraped {len(df_pop)} districts successfully.")
    display(df_pop[['district', 'population', 'pop_density']].head())

except Exception as e:
    print(f"❌ Error scraping: {e}")
    df_pop = pd.DataFrame()

# ---------------------------------------------------------
# 2. Merge with target_df
# ---------------------------------------------------------
if not df_pop.empty and 'target_df' in globals():
    print("\n2. Merging Population Data to Main Dataset...")

    # Clean ชื่อเขตใน df_pop ให้ตรงกับ target_df (ลบคำว่า "เขต")
    # ตัวอย่าง: "เขตพระนคร" -> "พระนคร"
    df_pop['district'] = df_pop['district'].astype(str).str.replace('เขต', '').str.strip()

    # Merge (Left Join)
    # ก่อน Merge ลบคอลัมน์เก่าออกก่อนถ้ามี (กันซ้ำ)
    if 'pop_density' in target_df.columns:
        target_df = target_df.drop(columns=['pop_density', 'population'])

    target_df = target_df.merge(df_pop[['district', 'population', 'pop_density']],
                                on='district',
                                how='left')

    # Fill Missing (สำหรับแถวที่ Join ไม่ติด)
    mean_density = target_df['pop_density'].mean()
    target_df['pop_density'] = target_df['pop_density'].fillna(mean_density)

    print("✅ Merge Completed!")
    print(target_df[['ticket_id', 'district', 'pop_density']].head())

else:
    print("❌ Cannot merge: Data not ready or 'target_df' missing.")

1. Scraping population data from Wikipedia...
   -> Raw columns: ['District (Khet)', 'Map Number', 'Postal code', 'Thai Name', 'Popu- lation', 'Number of Subdistricts (Khwaeng)', 'Latitude', 'Longitude']
   -> Scraped 50 districts successfully.


/tmp/ipython-input-2024734725.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


,district,population,pop_density
0,50,10150,0.096519
1,6,10240,0.068972
2,40,10160,0.052977
3,5,10220,0.053920
4,31,10120,0.106576



2. Merging Population Data to Main Dataset...
✅ Merge Completed!
     ticket_id  district  pop_density
0  2021-CGPMUN    ประเวศ          NaN
1  2021-7XATFA      สาทร          NaN
2  2021-DVEWYM  ลาดพร้าว          NaN
3  2021-8N9ZP8    ประเวศ          NaN
4  2021-8BTWZB    ประเวศ          NaN


In [ ]:
target_df.head()

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,organization_2,organization_3,dist_to_nearest_condo_km,nearest_condo_name,condo_count_1km,cluster,date,สภาพอากาศ,population,pop_density
0,1,2021-CGPMUN,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00:00,...,ฝ่ายโยธา เขตประเวศ,NaN,1.180890,South Park Apartment,0,1,2021-09-19,ฝนตก,NaN,NaN
1,2,2021-7XATFA,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00:00,...,NaN,NaN,0.134856,Silom Suite,19,1,2021-09-26,ฝนตก,NaN,NaN
2,4,2021-DVEWYM,ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00:00,...,ฝ่ายโยธา เขตลาดพร้าว,NaN,1.210388,Plum condo Chokchai4 (พลัมคอนโด โชคชัย4),0,1,2021-12-09,มีเมฆ,NaN,NaN
3,7,2021-8N9ZP8,คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 14:50:52.437512+00:00,...,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะอาดฯ เขตประเวศ,0.291524,Supalai Park (Condos),2,3,2021-12-18,มีเมฆ,NaN,NaN
4,9,2021-8BTWZB,ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00:00,...,ฝ่ายโยธา เขตประเวศ,NaN,1.457128,Supalai Park (Condos),0,3,2021-12-22,มีเมฆ,NaN,NaN


# scape ราคา condo/ตรม

In [1]:
import requests
import pandas as pd
import json
from typing import List, Dict
import time
import numpy as np

# Set up headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
}

# Mapping of district codes to district names
district_mapping = {
    'TH1050': 'บางบอน',
    'TH1006': 'บางกะปิ',
    'TH1040': 'บางแค',
    'TH1005': 'บางเขน',
    'TH1031': 'บางคอแหลม',
    'TH1021': 'บางขุนเทียน',
    'TH1047': 'บางนา',
    'TH1025': 'บางพลัด',
    'TH1004': 'บางรัก',
    'TH1029': 'บางซื่อ',
    'TH1020': 'บางกอกน้อย',
    'TH1016': 'บางกอกใหญ่',
    'TH1027': 'บึงกุ่ม',
    'TH1030': 'จตุจักร',
    'TH1035': 'จอมทอง',
    'TH1026': 'ดินแดง',
    'TH1036': 'ดอนเมือง',
    'TH1002': 'ดุสิต',
    'TH1017': 'ห้วยขวาง',
    'TH1043': 'คันนายาว',
    'TH1046': 'คลองสามวา',
    'TH1018': 'คลองสาน',
    'TH1033': 'คลองเตย',
    'TH1041': 'หลักสี่',
    'TH1011': 'ลาดกระบัง',
    'TH1038': 'ลาดพร้าว',
    'TH1010': 'มีนบุรี',
    'TH1003': 'หนองจอก',
    'TH1023': 'หนองแขม',
    'TH1007': 'ปทุมวัน',
    'TH1022': 'ภาษีเจริญ',
    'TH1014': 'พญาไท',
    'TH1009': 'พระโขนง',
    'TH1001': 'พระนคร',
    'TH1008': 'ป้อมปราบศัตรูพ่าย',
    'TH1032': 'ประเวศ',
    'TH1024': 'ราษฏร์บูรณะ',
    'TH1037': 'ราชเทวี',
    'TH1042': 'สายไหม',
    'TH1013': 'สัมพันธวงศ์',
    'TH1044': 'สะพานสูง',
    'TH1028': 'สาทร',
    'TH1034': 'สวนหลวง',
    'TH1019': 'ตลิ่งชัน',
    'TH1048': 'ทวีวัฒนา',
    'TH1015': 'ธนบุรี',
    'TH1049': 'ทุ่งครุ',
    'TH1045': 'วังทองหลาง',
    'TH1039': 'วัฒนา',
    'TH1012': 'ยานนาวา',
}

def fetch_ddproperty_all_districts(max_pages_per_district: int = 2) -> pd.DataFrame:
    """
    Scrape condo data from all Bangkok districts using districtCode
    
    Parameters:
    - max_pages_per_district: Maximum number of pages to scrape per district (default: 2)
    
    Returns:
    - DataFrame with columns: [ชื่อ condo, latitude, longitude, ราคา, ตารางเมตร, ราคาต่อตารางเมตร, เขต, ชื่อเขต]
    """
    
    all_listings = []
    base_url = "https://www.ddproperty.com/_next/data/SJdltGT3KUQnE_Gt6T1yQ/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2.json"
    
    district_codes = list(district_mapping.keys())
    total_districts = len(district_codes)
    
    for district_idx, district_code in enumerate(district_codes, 1):
        district_name = district_mapping[district_code]
        print(f"\n[{district_idx}/{total_districts}] Processing: {district_code} ({district_name})")
        
        for page in range(1, max_pages_per_district + 1):
            try:
                params = {
                    'page': page,
                    'districtCode': district_code,
                    'isCommercial': False,
                    'listingType': 'sale'
                }
                
                print(f"  Fetching page {page}...", end=' ')
                response = requests.get(base_url, params=params, headers=headers, timeout=10)
                response.raise_for_status()
                
                data = response.json()
                
                # Extract listings from the correct location in the API response
                page_data = data['pageProps'].get('pageData', {})
                listings_data = page_data.get('data', {})
                listings = listings_data.get('listingsData', [])
                
                print(f"({len(listings)} listings)")
                
                for listing in listings:
                    listing_data = listing.get('listingData', {})
                    
                    # Extract price per area from the pricePerArea field
                    price_per_area_str = listing_data.get('pricePerArea', {}).get('localeStringValue', '')
                    price_per_sqm = None
                    if price_per_area_str:
                        try:
                            price_per_sqm = float(price_per_area_str.split('฿')[1].split('/')[0].replace(',', '').strip())
                        except:
                            pass
                    
                    # Extract area (ตารางเมตร) from media carousel or other fields
                    area_sqm = None
                    # Try to extract from mediaCarousel if available
                    media_carousel = listing_data.get('mediaCarousel', {})
                    if isinstance(media_carousel, dict):
                        carousel_items = media_carousel.get('items', [])
                        for item in carousel_items:
                            if isinstance(item, dict) and 'label' in item:
                                label = item.get('label', '')
                                if 'ตร.ม' in label or 'ตารางเมตร' in label:
                                    try:
                                        area_sqm = float(label.split()[0].replace(',', ''))
                                    except:
                                        pass
                    
                    # Calculate price per sqm from price and area if not provided
                    if price_per_sqm is None and listing_data.get('price', {}).get('value') and area_sqm:
                        price_per_sqm = listing_data.get('price', {}).get('value') / area_sqm
                    
                    item = {
                        'ชื่อ condo': listing_data.get('localizedTitle'),
                        'latitude': np.nan,  # Not available in this API
                        'longitude': np.nan,  # Not available in this API
                        'ราคา': listing_data.get('price', {}).get('value'),
                        'ตารางเมตร': area_sqm,
                        'ราคาต่อตารางเมตร': price_per_sqm,
                        'เขต': district_code,
                        'ชื่อเขต': district_name,
                    }
                    
                    all_listings.append(item)
                
                # Be respectful - add delay between requests
                time.sleep(1)
                
            except requests.exceptions.RequestException as e:
                print(f"  Error: {e}")
                continue
    
    # Create DataFrame with specified column order
    df = pd.DataFrame(all_listings)
    
    # Ensure all columns exist in the correct order
    columns_order = ['ชื่อ condo', 'latitude', 'longitude', 'ราคา', 'ตารางเมตร', 'ราคาต่อตารางเมตร', 'เขต', 'ชื่อเขต']
    for col in columns_order:
        if col not in df.columns:
            df[col] = np.nan
    
    df = df[columns_order]
    
    return df

# Fetch the data
print("Starting to scrape DDProperty condo data from all Bangkok districts...")
print("This may take a few minutes...")
condo_df = fetch_ddproperty_all_districts(max_pages_per_district=2)

print(f"\n{'='*60}")
print(f"SUMMARY")
print(f"{'='*60}")
print(f"Total listings scraped: {len(condo_df)}")
print(f"\nDataFrame shape: {condo_df.shape}")
print(f"\nDataFrame columns: {list(condo_df.columns)}")

print(f"\n{'='*60}")
print("First 10 rows:")
print(condo_df.head(10))

print(f"\n{'='*60}")
print("Data types:")
print(condo_df.dtypes)

print(f"\n{'='*60}")
print("Statistics for numeric columns:")
print(condo_df[['ราคา', 'ตารางเมตร', 'ราคาต่อตารางเมตร']].describe())

print(f"\n{'='*60}")
print("Listings count by district:")
print(condo_df['ชื่อเขต'].value_counts().sort_index())

Starting to scrape DDProperty condo data from all Bangkok districts...
This may take a few minutes...

[1/50] Processing: TH1050 (บางบอน)
  Fetching page 1... (20 listings)
  Fetching page 2... (20 listings)

[2/50] Processing: TH1006 (บางกะปิ)
  Fetching page 1... (25 listings)
  Fetching page 2... (20 listings)


KeyboardInterrupt: 

In [ ]:
# Ensure 'ราคาต่อตารางเมตร' is numeric and coerce errors to NaN
condo_df['ราคาต่อตารางเมตร'] = pd.to_numeric(condo_df['ราคาต่อตารางเมตร'], errors='coerce')

# Filter out rows where 'ราคาต่อตารางเมตร' is NaN
df_condo_price_cleaned = condo_df.dropna(subset=['ราคาต่อตารางเมตร'])

# Group by 'เขต' (code) and 'ชื่อเขต' (name) and calculate the mean of 'ราคาต่อตารางเมตร'
df_avg_condo_price_per_district = df_condo_price_cleaned.groupby(['เขต', 'ชื่อเขต'])['ราคาต่อตารางเมตร'].mean().reset_index()

# Rename the column for clarity
df_avg_condo_price_per_district.rename(columns={'ราคาต่อตารางเมตร': 'avg_price_per_sqm'}, inplace=True)

print("Average condo price per square meter by district:")
display(df_avg_condo_price_per_district.head())

Average condo price per square meter by district:


,เขต,ชื่อเขต,avg_price_per_sqm
0,TH1001,พระนคร,105589.157895
1,TH1002,ดุสิต,153322.243243
2,TH1003,หนองจอก,23301.735294
3,TH1004,บางรัก,235432.555556
4,TH1005,บางเขน,75644.904762


In [ ]:
df_avg_condo_price_per_district.sort_values(by='avg_price_per_sqm', ascending=True)

,เขต,ชื่อเขต,avg_price_per_sqm
2,TH1003,หนองจอก,23301.735294
22,TH1023,หนองแขม,24743.057143
48,TH1049,ทุ่งครุ,31610.942857
49,TH1050,บางบอน,32888.060606
9,TH1010,มีนบุรี,38616.555556
20,TH1021,บางขุนเทียน,43563.026316
39,TH1040,บางแค,49583.742857
35,TH1036,ดอนเมือง,50569.135135
34,TH1035,จอมทอง,56076.783784
45,TH1046,คลองสามวา,59713.829268


In [ ]:
# Merge df_avg_condo_price_per_district into target_df
# Ensure column names are consistent for merging
target_df = pd.merge(target_df,
                     df_avg_condo_price_per_district[['ชื่อเขต', 'avg_price_per_sqm']],
                     left_on='district',
                     right_on='ชื่อเขต',
                     how='left')

# Drop the redundant 'ชื่อเขต' column from the merge
target_df.drop(columns=['ชื่อเขต'], inplace=True)

# Fill any NaN values in 'avg_price_per_sqm' with the mean, or 0 if a mean is not appropriate
# For now, let's fill with the overall mean or 0 if mean is NaN
if 'avg_price_per_sqm' in target_df.columns:
    mean_price = target_df['avg_price_per_sqm'].mean()
    target_df['avg_price_per_sqm'].fillna(mean_price if pd.notna(mean_price) else 0, inplace=True)

print("First 5 rows of target_df with new 'avg_price_per_sqm' column:")
display(target_df[['district', 'avg_price_per_sqm']].head())

First 5 rows of target_df with new 'avg_price_per_sqm' column:


/tmp/ipython-input-2651763244.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target_df['avg_price_per_sqm'].fillna(mean_price if pd.notna(mean_price) else 0, inplace=True)


,district,avg_price_per_sqm
0,ประเวศ,66629.444444
1,สาทร,190312.627907
2,ลาดพร้าว,85967.357143
3,ประเวศ,66629.444444
4,ประเวศ,66629.444444


In [ ]:
target_df.head()

,Unnamed: 0,ticket_id,comment,photo,photo_after,address,subdistrict,district,province,timestamp,...,organization_3,dist_to_nearest_condo_km,nearest_condo_name,condo_count_1km,cluster,date,สภาพอากาศ,population,pop_density,avg_price_per_sqm
0,1,2021-CGPMUN,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00:00,...,NaN,1.180890,South Park Apartment,0,1,2021-09-19,ฝนตก,NaN,NaN,66629.444444
1,2,2021-7XATFA,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00:00,...,NaN,0.134856,Silom Suite,19,1,2021-09-26,ฝนตก,NaN,NaN,190312.627907
2,4,2021-DVEWYM,ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00:00,...,NaN,1.210388,Plum condo Chokchai4 (พลัมคอนโด โชคชัย4),0,1,2021-12-09,มีเมฆ,NaN,NaN,85967.357143
3,7,2021-8N9ZP8,คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 14:50:52.437512+00:00,...,ฝ่ายรักษาความสะอาดฯ เขตประเวศ,0.291524,Supalai Park (Condos),2,3,2021-12-18,มีเมฆ,NaN,NaN,66629.444444
4,9,2021-8BTWZB,ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00:00,...,NaN,1.457128,Supalai Park (Condos),0,3,2021-12-22,มีเมฆ,NaN,NaN,66629.444444


In [ ]:
target_df.to_csv('scrape.csv', index=True, encoding='utf-8')